In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta

from datetime import datetime as dt
import itertools
from tqdm import tqdm
from scipy.stats import norm, t
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler,MinMaxScaler
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,log_loss
# import lightgbm as lgb
import gc

from sklearn.model_selection import StratifiedKFold,KFold
from sklearn import metrics
import xgboost as xgb


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_ori = pd.read_csv('/kaggle/input/airquoi/train.csv',parse_dates=['date'])
test_ori = pd.read_csv('/kaggle/input/airquoi/test.csv',parse_dates=['date'])
ss =pd.read_csv('/kaggle/input/airquoi/SampleSubmission.csv')

In [ ]:
# a = train.site_latitude.unique()
def get_prev(df,shifts = 1,col = 'pm2_5'):
    df['shifted_'+str(shifts)] = df.shift(shifts)[col].values
    return df
def target_encode(df_train,df_test,target,groupss):
    groupss_target_mean = df_train.groupby(groupss)[target].mean()
    groupss_target_max = df_train.groupby(groupss)[target].max()
    groupss_target_min = df_train.groupby(groupss)[target].min()
    groupss_target_std = df_train.groupby(groupss)[target].std()
    groupss_target_median = df_train.groupby(groupss)[target].apply(lambda x : np.median(x))
    groupss_target_q95 = df_train.groupby(groupss)[target].apply(lambda x : np.quantile(x,0.95))
    groupss_target_q05 = df_train.groupby(groupss)[target].apply(lambda x : np.quantile(x,0.05))
    groupss_target_skew = df_train.groupby(groupss)[target].skew()
    groupss_target_kurt = df_train.groupby(groupss)[target].apply(lambda x : x.kurtosis())
    # groupss_target_mean = df_train.groupby(groupss)[target].mean()
    df_test[groupss+target+'enc'] = df_test[groupss].map(groupss_target_mean)
    df_test[groupss+target+'enc'].fillna((df_test[groupss+target+'enc'].mean()), inplace=True)
    df_test[groupss+target+'enc1'] = df_test[groupss].map(groupss_target_max)
    df_test[groupss+target+'enc1'].fillna((df_test[groupss+target+'enc1'].mean()), inplace=True)
    df_test[groupss+target+'enc2'] = df_test[groupss].map(groupss_target_min)
    df_test[groupss+target+'enc2'].fillna((df_test[groupss+target+'enc2'].mean()), inplace=True)
    df_test[groupss+target+'enc3'] = df_test[groupss].map(groupss_target_std)
    df_test[groupss+target+'enc3'].fillna((df_test[groupss+target+'enc3'].mean()), inplace=True)
    df_test[groupss+target+'enc4'] = df_test[groupss].map(groupss_target_median)
    df_test[groupss+target+'enc4'].fillna((df_test[groupss+target+'enc4'].mean()), inplace=True)
    df_test[groupss+target+'enc5'] = df_test[groupss].map(groupss_target_q95)
    df_test[groupss+target+'enc5'].fillna((df_test[groupss+target+'enc5'].mean()), inplace=True)
    df_test[groupss+target+'enc6'] = df_test[groupss].map(groupss_target_q05)
    df_test[groupss+target+'enc6'].fillna((df_test[groupss+target+'enc6'].mean()), inplace=True)
    df_test[groupss+target+'enc7'] = df_test[groupss].map(groupss_target_skew)
    df_test[groupss+target+'enc7'].fillna((df_test[groupss+target+'enc7'].mean()), inplace=True)
    df_test[groupss+target+'enc8'] = df_test[groupss].map(groupss_target_kurt)
    df_test[groupss+target+'enc8'].fillna((df_test[groupss+target+'enc8'].mean()), inplace=True)
    tmp =np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].apply(lambda x : x.skew())
        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc7'] = tmp
    tmp =np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].apply(lambda x : x.kurtosis())

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc8'] = tmp
    
    tmp =np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].apply(lambda x : np.quantile(x,0.05))

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc6'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991032)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].apply(lambda x : np.quantile(x,0.95))

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc5'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991035)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].apply(lambda x : np.median(x))

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc4'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby(groupss)[target].mean()

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_mean)
    df_train[groupss+target+'enc'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991025)
    for idx_1, idx_2 in kf.split(df_train):
        target_max = df_train.iloc[idx_1].groupby(groupss)[target].max()

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_max)
    df_train[groupss+target+'enc1'] = tmp
    # from sklearn.model_selection import KFold/
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1990325)
    for idx_1, idx_2 in kf.split(df_train):
        target_min = df_train.iloc[idx_1].groupby(groupss)[target].min()

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_min)
    df_train[groupss+target+'enc2'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_std = df_train.iloc[idx_1].groupby(groupss)[target].std()

        tmp[idx_2] = df_train[groupss].iloc[idx_2].map(target_std)
    df_train[groupss+target+'enc3'] = tmp
    df_train[groupss+target+'enc'].fillna((df_train[groupss+target+'enc'].mean()), inplace=True)
    df_train[groupss+target+'enc1'].fillna((df_train[groupss+target+'enc1'].mean()), inplace=True)
    df_train[groupss+target+'enc2'].fillna((df_train[groupss+target+'enc2'].mean()), inplace=True)
    df_train[groupss+target+'enc3'].fillna((df_train[groupss+target+'enc3'].mean()), inplace=True)
    df_train[groupss+target+'enc4'].fillna((df_train[groupss+target+'enc4'].mean()), inplace=True)
    df_train[groupss+target+'enc5'].fillna((df_train[groupss+target+'enc5'].mean()), inplace=True)
    df_train[groupss+target+'enc6'].fillna((df_train[groupss+target+'enc6'].mean()), inplace=True)
    return df_train, df_test

In [ ]:
def preprocess(train,test):
    le = LabelEncoder()
    train['Train']=1
    test['Train']=0
#     home_teams=train['Home Team'].unique()
    all_data=pd.concat([train,test])
    all_data['month'] = all_data.date.dt.month
    all_data['year'] = all_data.date.dt.year
    all_data['day'] = all_data.date.dt.day
    all_data['week'] = all_data.date.dt.week
    all_data['dow'] = all_data.date.dt.dayofweek
    all_data['woy'] = all_data.date.dt.weekofyear
    all_data[['site_latitude','site_longitude']] =  all_data[['site_latitude','site_longitude']] .round(5)
#     for c in all_data.drop(['date','pm2_5'], axis = 1).select_dtypes(exclude = 'object').columns:
#         all_data[c] = all_data[c].fillna(all_data[c].min()-1)
#     all_data['rel_date'] = all_data.groupby('site_latitude').apply(lambda x : ((x.date-x.date.min())/ np.timedelta64(1, 'D')).astype('int') +1).reset_index()['date']

    all_data['device'] = le.fit_transform(all_data['device'].values.reshape(-1,1))
    le = LabelEncoder()
#     all_data['site_code'] = le.fit_transform(all_data.site_latitude.astype('str').values.reshape(-1,1))
#     all_data = get_prev(all_data.reset_index(drop = True),shifts = 1,col = 'pm2_5')
    train = all_data.loc[all_data.Train == 1].reset_index(drop = True)
    test = all_data.loc[all_data.Train == 0].reset_index(drop = True)
#     train, test = interpol(train,test)
#     train,test = target_encode(train,test,target = 'pm2_5',groupss = 'site_latitude')
    return train, test
def _get_X_Y_DF_from_CV(train_X, train_Y, train_index, validation_index):
        X_train, X_validation = (
            train_X.iloc[train_index],
            train_X.iloc[validation_index],
        )
        y_train, y_validation = (
            train_Y.iloc[train_index],
            train_Y.iloc[validation_index],
        )
        return X_train, X_validation, y_train, y_validation
    

def train_lgb(X,y,test_data,params,seed_lgb,shuffle_lgb):
    features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
    models =[]
    train_X = X.copy()
    train_Y = y.copy()
    test_X = test_data.copy()
    print(f"Shape of train_X : {train_X.shape}, test_X: {test_X.shape}, train_Y: {train_Y.shape}")

    predictors = list(train_X.columns)
    # print(f"List of features to be used {list(predictors)}")

    # Selecting n_splits to be 3, since class 42 has 
    # just 3 instances
    kf = KFold(random_state=seed_lgb,n_splits=K_FOLDS, shuffle=shuffle_lgb)
    y_oof_lgb = np.zeros(shape=(len(train_X),))
    y_predicted_lgb = np.zeros(shape=(len(test_X),))
    cv_scores = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=train_Y):
        fold += 1
        print(f"fold {fold} of {n_folds}")

        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

        model = lgb.train(
            lgb_params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=100,
            early_stopping_rounds=EARLY_STOPPING_ROUNDS,
            num_boost_round=N_ESTIMATORS,
            feature_name=predictors,
            categorical_feature="auto",
        )
        del lgb_train, lgb_eval, train_index, X_train, y_train
        gc.collect()

        y_oof_lgb[validation_index] = model.predict(
            X_validation, num_iteration=model.best_iteration
        )

        y_predicted_lgb += model.predict(
            test_data.values, num_iteration=model.best_iteration
        )
        fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
        fold_importance_df['Feature']= predictors
        fold_importance_df['Importance']= model.feature_importance()
        fold_importance_df["fold"] = fold + 1
        features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
        models.append(model)

        best_iteration = model.best_iteration
        print(f"Best number of iterations for fold {fold} is: {best_iteration}")

        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof_lgb[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted_lgb /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof_lgb), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted_lgb,models,y_oof_lgb,oof_score,features_importance
def train_catbo(train_X, train_Y,test_X,params,seed_cat,shuffle_cat):
# model.fit(train_pool,plot=True,eval_set=test_pool)
    kf = KFold(random_state=seed_cat,n_splits=K_FOLDS, shuffle=shuffle_cat)
    y_oof = np.zeros(shape=(len(train_X),))
    y_predicted = np.zeros(shape=(len(test_X),))
    cv_scores = []
    models = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=train_Y):
        fold += 1
        print(f"fold {fold} of {n_folds}")

        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )

        train_pool = Pool(data=X_train, label=y_train)
        eval_pool = Pool(data=X_validation, label=y_validation.values) 
        model = CatBoostRegressor(**params)
        model.fit(train_pool,plot=True,eval_set=eval_pool)
        del train_index, X_train, y_train
        gc.collect()
        models.append(model)
        y_oof[validation_index] = model.predict(
            X_validation )

        y_predicted += model.predict(
            test_X.values
        )

    #     best_iteration = model.best_iteration
    #     print(f"Best number of iterations for fold {fold} is: {best_iteration}")

        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score
def train_xgb(X,y,test_data,params,num_iter=1500,es = 100,ve = 100,seed_xgb=42,shuffle_xgb=True):
    features = X.columns
    X = X.values
    # y = train['target'].values
    y_oof = np.zeros(shape=(len(X),))
    y_predicted = np.zeros(shape=(len(test_data), ))
    cv_scores = []
    models = []
    kf = KFold(random_state=seed_xgb,n_splits=K_FOLDS, shuffle=shuffle_xgb)

    for i, (train_index, test_index) in enumerate(kf.split(X, y)):
        print(' xgb kfold: {}  of  {} : '.format(i+1, K_FOLDS ))
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        d_train = xgb.DMatrix(X_train, y_train) 
        d_valid = xgb.DMatrix(X_valid, y_valid) 
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        xgb_model = xgb.train(params, d_train, num_iter, watchlist,
                              early_stopping_rounds=es, 
                            verbose_eval=ve)
        models.append(xgb_model)
        y_oof[test_index] = xgb_model.predict(xgb.DMatrix(X_valid), 
                            ntree_limit=xgb_model.best_ntree_limit)
        y_predicted += xgb_model.predict(xgb.DMatrix(test_data[features].values), 
                            ntree_limit=xgb_model.best_ntree_limit) 
        
        cv_oof_score = metrics.mean_absolute_error(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

#         del validation_index, X_validation, y_validation
#         gc.collect()

    y_predicted /= K_FOLDS
    oof_score = round(metrics.mean_absolute_error(y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 
def train_keras(X,train,test_data):
#     train ['Original_score'] = le.inverse_transform(train.Score)
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     X_scaled = scaler.fit_transform(X)
#     X_scaled = pd.DataFrame(X_scaled)
#     Y = pd.get_dummies(train['Original_score'])
    X_scaled = X.values
    Y = Y.values
    inp = len(X.columns)
    y_oof = np.zeros(shape=(len(X), NUM_CLASSES))
    y_predicted = np.zeros(shape=(len(test_data), NUM_CLASSES))
    cv_scores = []
    models = []
    kf = KFold(random_state=SEED,n_splits=K_FOLDS, shuffle=True)
    for i, (train_index, test_index) in enumerate(kf.split(X_scaled, Y)):
        print(' keras kfold: {}  of  {} : '.format(i+1, K_FOLDS ))
        X_train, X_valid = X_scaled[train_index], X_scaled[test_index]
        y_train, y_valid = Y[train_index], Y[test_index]
#         np.random.seed(SEED)
        my_model = baseline_model(inp)
        my_model.fit(X_train, y_train,
                     validation_data=(X_valid, y_valid),
                     epochs=1000,
                     callbacks=[EarlyStopping(patience=20)],
                     verbose=0)
        
        models.append(my_model)
        y_oof[test_index] = my_model.predict(X_valid)
        y_predicted += my_model.predict(test_data.values) 
        del my_model
        gc.collect()
        cv_oof_score = metrics.log_loss(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

#         del validation_index, X_validation, y_validation
#         gc.collect()

    y_predicted /= K_FOLDS
    oof_score = round(metrics.log_loss(y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 
from matplotlib import pyplot as plt
# Keras imports
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import EarlyStopping
import os 
import random
import numpy as np 

DEFAULT_RANDOM_SEED = 2021

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# tensorflow random seed 
import tensorflow as tf 
def seedTF(seed=DEFAULT_RANDOM_SEED):
    tf.random.set_seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTF(seed)
    seedTorch(seed)
seedEverything(42)
seed = 42

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:10].index
    best_features = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:50]
    best_features.reset_index(inplace=True)
    print(best_features.dtypes)
    plt.figure(figsize=(8, 10))
    sns.barplot(x="Importance", y="Feature", data=best_features)
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
def preds_to_sub(test,preds,name = None,log = False):
    if log :
        test['pm2_5'] = np.exp(preds)
    else:
        test['pm2_5'] = preds
    sub = test[['ID','pm2_5']]
    if name:
        sub.to_csv(name+'.csv', index = False)
    return sub

In [ ]:
# distances[4].sort_values()
def get_dist(all_data, i,j):
    return np.sqrt((all_data.loc[all_data.site_code == i,'site_latitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_latitude'].unique()[0])**2 + (all_data.loc[all_data.site_code == i,'site_longitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_longitude'].unique()[0])**2)
def interpol(train,test):
    train.sort_values(by = 'date',inplace = True)
    train.index = train.date
    train = train.drop('date',axis = 1).sort_index().groupby('site_code').resample('D').mean().reset_index(drop = True,level = 0).reset_index()
    cols = train.drop('date',axis = 1).select_dtypes(exclude= 'object').columns
    train[cols]= train[cols].interpolate()
    test.sort_values(by = 'date',inplace = True)
    test_0 = test[['ID','month','year','day','site_code']].copy()
    test.index = test.date
    test = test.drop('date',axis = 1).sort_index().groupby('site_code').resample('D').mean().reset_index(drop = True,level = 0).reset_index()
    cols = test.drop('date',axis = 1).select_dtypes(exclude= 'object').columns
    test[cols] = test[cols].interpolate()
    test = test_0.merge(test[cols],on = ['month','year','day','site_code'],how = 'left')
#     test = test[~test.ID.isna()]
    return train, test
def compare(i,j,train):
    a = train.loc[train.site_code == i].sort_values(by = 'date')
    a.index = a.date
    a = a['pm2_5'].resample('D').mean()
    b = train.loc[train.site_code == j].sort_values(by= 'date')
    b.index = b.date
    b = b['pm2_5'].resample('D').mean()
    min_idx = max(a.index.min(),b.index.min())
    max_idx = min(a.index.max(),b.index.max())
    b = b[min_idx:max_idx]
    a = a[min_idx:max_idx]
    a = a.fillna(b)
    b = b.fillna(a)
    a = a.dropna()
    b = b.dropna()
    print(np.corrcoef(a,b))
    print(mae(b[min_idx:max_idx].values,a[min_idx:max_idx].values))
    plt.plot(a[min_idx:max_idx])
    plt.plot(b[min_idx:max_idx],alpha = 0.5)

In [ ]:

train,test = preprocess(train_ori,test_ori)

In [ ]:
# all_data = pd.concat([train,test])
# s_codes = all_data.site_code.unique()
# distances = np.zeros((s_codes.shape[0],s_codes.shape[0]))
# for i in np.sort(s_codes):
#     for j in np.sort(s_codes):
#         distances[i,j] =  get_dist(all_data, i,j)
# distances = pd.DataFrame(index = np.sort(s_codes), columns = np.sort(s_codes),data = distances)

In [ ]:
SO2_cols = ['SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_solar_zenith_angle',
       'SulphurDioxide_SO2_column_number_density_15km']
CO_cols = ['CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
       'CarbonMonoxide_sensor_azimuth_angle',
       'CarbonMonoxide_sensor_zenith_angle',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle']
NO2_cols = ['NitrogenDioxide_NO2_column_number_density',
       'NitrogenDioxide_tropospheric_NO2_column_number_density',
       'NitrogenDioxide_stratospheric_NO2_column_number_density',
       'NitrogenDioxide_NO2_slant_column_number_density',
       'NitrogenDioxide_tropopause_pressure',
       'NitrogenDioxide_absorbing_aerosol_index',
       'NitrogenDioxide_cloud_fraction', 'NitrogenDioxide_sensor_altitude',
       'NitrogenDioxide_sensor_azimuth_angle',
       'NitrogenDioxide_sensor_zenith_angle',
       'NitrogenDioxide_solar_azimuth_angle',
       'NitrogenDioxide_solar_zenith_angle']
HCHO_cols = ['Formaldehyde_tropospheric_HCHO_column_number_density',
       'Formaldehyde_tropospheric_HCHO_column_number_density_amf',
       'Formaldehyde_HCHO_slant_column_number_density',
       'Formaldehyde_cloud_fraction', 'Formaldehyde_solar_zenith_angle',
       'Formaldehyde_solar_azimuth_angle', 'Formaldehyde_sensor_zenith_angle',
       'Formaldehyde_sensor_azimuth_angle']
UV_cols = ['UvAerosolIndex_absorbing_aerosol_index',
       'UvAerosolIndex_sensor_altitude', 'UvAerosolIndex_sensor_azimuth_angle',
       'UvAerosolIndex_sensor_zenith_angle',
       'UvAerosolIndex_solar_azimuth_angle',
       'UvAerosolIndex_solar_zenith_angle']
O3_cols = ['Ozone_O3_column_number_density_amf',
       'Ozone_O3_slant_column_number_density',
       'Ozone_O3_effective_temperature', 'Ozone_cloud_fraction',
       'Ozone_sensor_azimuth_angle', 'Ozone_sensor_zenith_angle',
       'Ozone_solar_azimuth_angle', 'Ozone_solar_zenith_angle']
cloud_cols = ['Cloud_cloud_fraction', 'Cloud_cloud_top_pressure',
       'Cloud_cloud_top_height', 'Cloud_cloud_base_pressure',
       'Cloud_cloud_base_height', 'Cloud_cloud_optical_depth',
       'Cloud_surface_albedo', 'Cloud_sensor_azimuth_angle',
       'Cloud_sensor_zenith_angle', 'Cloud_solar_azimuth_angle',
       'Cloud_solar_zenith_angle']
target = ['pm2_5']
other_cols = ['pm2_5', 'Train', 'month', 'year', 'day',
       'week', 'dow', 'woy', 'rel_date']
other_cols_test = ['Train', 'month', 'year', 'day',
       'week', 'dow', 'woy', 'rel_date']

In [ ]:
drop_cols = ['ID','date','Train','pm2_5']
drop_cols2 = ['ID','date','shifted_1',
            'month', 'year', 'day',
       'week', 'dow', 'woy', 'shifted_1', 'site_latitudepm2_5enc7',
       'site_latitudepm2_5enc8', 'site_latitudepm2_5enc6',
       'site_latitudepm2_5enc5', 'site_latitudepm2_5enc4',
       'site_latitudepm2_5enc', 'site_latitudepm2_5enc1',
       'site_latitudepm2_5enc2', 'site_latitudepm2_5enc3']

In [ ]:
y = train['pm2_5']

In [ ]:

SEED = 42
NUM_CLASSES = 3
EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 5000
K_FOLDS = 5
lgb_params = {
    "objective": "regression",
    "boosting_type": "gbdt",
    "learning_rate": 0.01,
    "n_jobs": 4,
    "seed": SEED,
    "max_depth": 3,
#     "max_bin": 255,
#     'reg_lambda': 0.1,  # L1 regularization term on weights
#     'reg_lambda': 1,
    "metric": "mae",
    "verbose": -1,
}
y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train.drop([d for d in drop_cols if d in train.columns],axis = 1),
                                                                                       y,test.drop([d for d in drop_cols if d in test.columns],axis = 1),
                                                                                       lgb_params,None,False)

In [ ]:
def cust_score(y_predicted):
    l = []
    a = train.site_latitude.unique()
    for i in a:
        y_true = train.loc[train.site_latitude == i,'pm2_5']
        y_hat = y_predicted[y_true.index]
        l.append(metrics.mean_absolute_error(y_true,y_hat))
    return pd.DataFrame({'site_latitude':a,
                        'err': np.array(l)}).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()

In [ ]:
cust_score(y_oof_lgb)

In [ ]:
from catboost import Pool, CatBoostRegressor
params_cat = {'iterations':1500,
        'learning_rate':0.01,
        'random_strength':0.1,
        'early_stopping_rounds':30, 
        'depth':3,
        'loss_function':'RMSE',
        'eval_metric':'MAE',
        'verbose' : 100,
        'leaf_estimation_method':'Newton'}
y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train.drop([d for d in drop_cols if d in train.columns],axis = 1),
                                                                        y,test.drop([d for d in drop_cols if d in test.columns],axis = 1),params_cat,
                                                                seed_cat = None, shuffle_cat = False)

In [ ]:
cust_score(y_oof_cat)

In [ ]:
# anal(4,y_predicted_lgb,y_oof_lgb)

In [ ]:
params_xgb = {"objective":"reg:squarederror",'learning_rate': 0.01,
           'max_depth': 3}#, 'subsample': 0.9,
#           'colsample_bytree': 0.9}

y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train.drop([d for d in drop_cols if d in train.columns],axis = 1),
                                                                        y,test.drop([d for d in drop_cols if d in test.columns],axis = 1),
                                                             params_xgb,num_iter=1000,es = 30,ve = 100)

In [ ]:
cust_score(y_oof_xgb)

In [ ]:
train['preds_lgb'] = y_oof_lgb                                                                                 
train['preds_xgb'] = y_oof_xgb
train['preds_cat'] = y_oof_cat                                                                                 
# train['preds_keras'] = y_oof_keras
test['preds_lgb'] = y_predicted_lgb                                                                                 
test['preds_xgb'] = y_predicted_xgb
test['preds_cat'] = y_predicted_cat                                                                                 
# test['preds_keras'] = y_predicted_keras

In [ ]:
# i = 0
# a = train.site_latitude.unique()
# df_train = train.groupby('site_latitude').get_group(a[i]).sort_values(by = 'date')
# df_test = test.groupby('site_latitude').get_group(a[i]).sort_values(by = 'date')
# df_train.index = df_train.date
# df_test.index = df_test.date
# ser_true_train = df_train['pm2_5']
# ser_lgb_train = df_train['preds_lgb']
# ser_xgb_train = df_train['preds_xgb']
# ser_cat_train = df_train['preds_cat']
# ser_lgb_test = df_test['preds_lgb']
# ser_xgb_test = df_test['preds_xgb']
# ser_cat_test = df_test['preds_cat']

# plt.plot(ser_true_train[-100:])
# plt.plot(ser_lgb_train[-100:])
# plt.plot(ser_xgb_train[-100:])
# plt.plot(ser_cat_train[-100:])
# plt.plot(ser_lgb_test)
# plt.plot(ser_xgb_test)
# plt.plot(ser_cat_test)

In [ ]:
# # single models:
# preds_to_sub(test,y_predicted_cat,'over_cat')
# preds_to_sub(test,y_predicted_lgb,'over_lgb')
# preds_to_sub(test,y_predicted_xgb,'over_xgb')

In [ ]:
preds_to_sub(test,((y_predicted_cat) + (y_predicted_lgb) + (y_predicted_xgb) )/3)

In [ ]:
# display_importances(features_importance_lgb)

In [ ]:
### Analyee errors

In [ ]:
train['preds'] = ((y_oof_lgb)+(y_oof_xgb)+(y_oof_cat))/3
train['errors'] = train['pm2_5'] - train['preds']
train['abs_errors'] = abs(train['errors'])

In [ ]:
!pip install chart-studio

In [ ]:
# plt.pl
# add_cols
# Importing libraries
a = train.site_latitude.unique()
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error

In [ ]:
test['correct'] = 0
train['correct'] = 0

In [ ]:
def masked_w_aver(a,weights):
    ma = np.ma.MaskedArray(a, mask=np.isnan(a))
    return np.ma.average(ma, weights=weights)
def auto_regress(train,test,i):
    time_ser = fancy_interpol(train,i,th=0.8,col_pred = 'pm2_5',col_tar = 'errors')
    time_ser = time_ser.resample('D').mean().interpolate()
    time_ser_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date').copy()
    time_ser_test.index = time_ser_test.date 
    time_ser_test = (time_ser_test['correct'])
    time_ser_test = time_ser_test.resample('D').mean().interpolate()

    train_sample = time_ser.values
    p_val = 0.05
    try:
        seas  = 1
        model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),enforce_stationarity=True,
                       seasonal_order=(0,0,1,7),trend= None)
        result = model.fit(maxiter=1000,disp=False)
        assert result.pvalues[2]<p_val
    except:
        seas = 0
        model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),trend=None)
        result = model.fit(maxiter=1000,disp=False)
#     print(result.summary())
#     predicted_result = result.predict(start=1, end=train_sample.shape[0]-1)
    y_true_dates = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['date'].values
    y_true = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['pm2_5'].values
    y_hat = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['preds'].values
    correct = result.predict(start=1, end=time_ser.shape[0]+time_ser_test.shape[0])
    y_true_dates_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date')['date'].values
    mask_train = time_ser.index.isin(y_true_dates)
    mask_test = time_ser_test.index.isin(y_true_dates_test)
    train.loc[train.site_latitude == a[i],'correct'] = correct[:time_ser.shape[0]][mask_train]
    test.loc[test.site_latitude == a[i],'correct'] = correct[time_ser.shape[0]:][mask_test]
    train.loc[train.site_latitude == a[i],'seas'] = seas
    test.loc[test.site_latitude == a[i],'seas'] = seas
    return train, test
def fancy_interpol(train,i,th=0.8,col_pred = 'pm2_5',col_tar = 'pm2_5'):
#     th = 0.8
    mi = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.min()
    ma = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.max()

    df_pred = train.pivot(index = 'date',columns = 'site_latitude',values = col_pred)[mi:ma]
    df_tar = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[mi:ma]

    w_s = df_pred.corr()[a[i]].sort_values(ascending = False)
    filtered_ws = w_s[(w_s>th)& (w_s<1)]
    if len(filtered_ws)*len(df_tar[df_tar[a[i]].isna()])>0 : 
        df_tar.loc[df_tar[a[i]].isna(),a[i]] = df_tar.loc[df_tar[a[i]].isna(),filtered_ws.index].apply(lambda x: masked_w_aver(x, weights=filtered_ws.values),axis = 1)
    return df_tar.loc[mi:ma,a[i]].astype('float')
#     print(metrics.mean_absolute_error(y_true,y_hat))
#     print(metrics.mean_absolute_error(y_true,y_hat+correct[mask_train]))

In [ ]:
train['correct']= 0
test['correct'] = 0
for i in range(a.shape[0]):
    train,test = auto_regress(train,test,i)

In [ ]:
train['refined'] = train['correct'] + train['preds']

In [ ]:
df_val = train.groupby('site_latitude').apply(lambda x :x.sort_values('date').iloc[-150:]).reset_index(drop = True)

In [ ]:
# metrics.mean_absolute_error(df_val['pm2_5'],df_val['refined'])
# train['seas']
test['pm2_5'] = test['pm2_5']+test['correct']

In [ ]:
time_ser.resample('W').mean()

In [ ]:
# metrics.mean_absolute_error(train.pm2_5,train.preds+train.correct)
pd.DataFrame({'ID': test.ID.sort_index(),
    'pm2_5':(test.pm2_5 + test.correct).sort_index()}).to_csv('sub_sar.csv',index = False)

In [ ]:
tr[tr.correct!=0].site_latitude.unique()

In [ ]:
# len(filtered_ws)
# assert False

In [ ]:
train_cop = train.copy()
test_cop = test.copy()

In [ ]:
train = train_cop.copy()
test = test_cop.copy()

In [ ]:
# test[(test['correct'] != 0)]
col_tar = 'errors'
col_pred = 'pm2_5'
th = 0.8
mi = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.min()
ma = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.max()

df_pred = train.pivot(index = 'date',columns = 'site_latitude',values = col_pred)[mi:ma]
df_tar = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[mi:ma]

w_s = df_pred.corr()[a[i]].sort_values(ascending = False)
filtered_ws = w_s[(w_s>th)& (w_s<1)]

In [ ]:
# filtered_ws
# a
# df_tar.loc[df_tar[a[i]].isna(),a[i]] = df_tar.loc[df_tar[a[i]].isna(),filtered_ws.index].apply(lambda x: masked_w_aver(x, weights=filtered_ws.values),axis = 1)
# len(df_tar[df_tar[a[i]].isna()])
train.columns

In [ ]:
# train = train.sort_values(by = 'date')
# time_ser = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')
# time_ser.index = time_ser.date
# time_ser = (time_ser['errors'])
# time_ser = time_ser.resample('D').mean().interpolate()
# test['correct'] = 0
# train['correct'] = 0
# time_ser_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date')
# time_ser_test.index = time_ser_test.date 
# time_ser_test = (time_ser_test['correct'])
# time_ser_test = time_ser_test.resample('D').mean().interpolate()

# train_sample = time_ser.values

# model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),trend=None)
# result = model.fit(maxiter=1000,disp=False)
# #     print(result.summary())
# #     predicted_result = result.predict(start=1, end=train_sample.shape[0]-1)
# y_true_dates = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['date'].values
# y_true = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['pm2_5'].values
# y_hat = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['preds'].values
# correct = result.predict(start=1, end=time_ser.shape[0]+time_ser_test.shape[0])
# y_true_dates_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date')['date'].values
# mask_train = time_ser.index.isin(y_true_dates)
# mask_test = time_ser_test.index.isin(y_true_dates_test)
# train.loc[train.site_latitude == a[i],'correct'] = correct[:time_ser.shape[0]][mask_train]
# test.loc[test.site_latitude == a[i],'correct'] = correct[time_ser.shape[0]:][mask_test]

In [ ]:
# time_ser
# correct[:time_ser.shape[0]][mask_train].shape
# test.loc[test.site_latitude == a[i],'correct'].shape
# correct[time_ser.shape[0]:].shape

In [ ]:
# tr[tr.correct!=0]

In [ ]:
# time_ser.isna().sum()
# time_ser = time_ser.resample('D').mean().interpolate()
# time_ser.isna().sum()

In [ ]:
# sm.tsa.season.al_decompose(time_ser)
time_ser.isna().any()

In [ ]:
# time_ser
fancy_interpol(train,6,th=0.3,col_tar = 'errors',col_pred = 'pm2_5')

In [ ]:
# time_ser = train.loc[train.site_latitude == a[3]].sort_values(by = 'date')
# time_ser.index = time_ser.date
# time_ser = (time_ser['errors'])
i = 3
q = train['pm2_5'].quantile(0.99)

train_q = train[train.pm2_5<q].reset_index(drop = True )
time_ser = fancy_interpol(train_q,i,th=0.8,col_tar = 'errors',col_pred = 'pm2_5')
time_ser = time_ser.resample('D').mean().interpolate()
rcParams['figure.figsize'] = 11, 9
decomposed_google_volume = sm.tsa.seasonal_decompose(time_ser) # The frequncy is annual
figure = decomposed_google_volume.plot()
plt.show()
# correct[time_ser.shape[0]:][mask_test].shape
# mask_test

In [ ]:
# time_ser

In [ ]:
# predicted_result.shape
# a= train.site_latitude.unique()

In [ ]:
# df.loc[df[a[i]].isna(),a[i]]
# w_s
# df.loc[df[a[i]].isna(),w_s[(w_s>th)& (w_s<1)].index]
len(w_s[(w_s>th)& (w_s<1)])

In [ ]:
# fancy_interpol(train,0,th=0.8,col_tar = 'errors',col_pred = 'pm2_5')
# fancy_interpol(train,i,th=0.8,col_tar = 'errors',col_pred = 'pm2_5')

In [ ]:
time_ser

In [ ]:
# time_ser.isna()

In [ ]:
i = 1
alpha = 0.05
q_up = train['pm2_5'].quantile(1-alpha)
q_low = train['pm2_5'].quantile(alpha)
train_q = train[(train.pm2_5<q_up)].reset_index(drop = True )

time_ser = fancy_interpol(train_q,i,th=0.9,col_tar = 'errors',col_pred = 'pm2_5')
time_ser = time_ser.resample('D').mean().interpolate()

train_sample = time_ser.iloc[:-150].values

model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),enforce_stationarity=True,
                       seasonal_order=(0,0,1,7),trend= None)
result = model.fit(maxiter=1000,disp=False)
print(result.summary())
predicted_result = result.predict(start=1, end=time_ser.shape[0])
result.plot_diagnostics()
# calculating error
rmse = math.sqrt(mean_squared_error(time_ser[0:time_ser.shape[0]+1], predicted_result))
print("The root mean squared error is {}.".format(rmse))

In [ ]:
# train.columns

In [ ]:
y_correct = pd.Series(index = time_ser.index,data = predicted_result)[train_q.loc[train_q.site_latitude == a[i]].sort_values(by = 'date')['date'].values].values
y_preds = train_q.loc[train_q.site_latitude == a[i]].sort_values(by = 'date')['preds']
y_true = train_q.loc[train_q.site_latitude == a[i]].sort_values(by = 'date')['pm2_5'].values
metrics.mean_absolute_error(y_true,(y_preds + y_correct))

In [ ]:
metrics.mean_absolute_error(y_true[-150:],(y_preds[-150:] + y_correct[-150:]))

In [ ]:
metrics.mean_absolute_error(y_true,y_preds)

In [ ]:
# y_hat.shape[0]
# train
# train.pivot(index = 'date',columns = 'site_latitude',values = 'errors').corr()[a[0]].sort_values(ascending = False,key  = abs)

In [ ]:
# train.pivot(index = 'date',columns = 'site_latitude',values = 'pm2_5').corr()[a[0]].sort_values(ascending = False,key  = abs)

In [ ]:
# np.sqrt(((time_ser[1:-1] - np.cumsum(predicted_result))**2).mean())
# mean_absolute_error()
i = 4

In [ ]:
# result.predict(time_ser.index)
# correct[]

In [ ]:
# a = train.site_latitude.unique()
# train.reset_index(drop = True, inplace = True)
plot_acf(time_ser,lags=25,title="PM2_5 Err")
plt.show()

In [ ]:
adf = adfuller(time_ser)
print("p-value of pm2_5: {}".format(float(adf[1])))
# adf = adfuller(google["Volume"])
# print("p-value of google: {}".format(float(adf[1])))

In [ ]:
decomposed_google_volume.trend.plot()

In [ ]:
plt.plot(train.groupby('site_latitude').get_group(a[0]).sort_values(by = 'date')['preds'].values)
plt.plot(train.groupby('site_latitude').get_group(a[0]).sort_values(by = 'date')['pm2_5'].values,alpha = 0.2)
# train

In [ ]:
# train['errors'] = train['preds'] - train['pm2_5']
(train.date.max()- train.date.min()).days

In [ ]:
# train.drop(drop_cols2, axis = 1)[abs(train['preds'] - train['pm2_5'])>15].corr()['pm2_5'].sort_values(ascending = False, key = abs)

In [ ]:
X_

In [ ]:
# X_train = train.loc[(train.site_laritude == a[0])]
# lr.fit()
def get_train_val(train,test_size,group):
    temp = train.loc[(train.site_latitude == group)]
    X_train = temp.loc[temp.date < temp.date.min() + (1-test_size)*(temp.date.max()-temp.date.min()) ]
    X_val = temp.loc[temp.date >= temp.date.min() + (1-test_size)*(temp.date.max()-temp.date.min())]
    return X_train, X_val

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
def fit_line(group,test_size):
    X_train, X_val = get_train_val(train.fillna(0),test_size,group)
    lr = LinearRegression()
    lr.fit(X_train.drop(drop_cols, axis = 1),np.log(X_train['pm2_5']))
    preds = np.exp(lr.predict(X_val.drop(drop_cols, axis = 1)))
    plt.plot(X_val['pm2_5'].values,label = 'True')
    plt.plot(preds,label = 'preds')
    plt.axhline(y = X_train['pm2_5'].mean())
    plt.legend()
    plt.show()
    print(mae(X_val['pm2_5'],preds))

In [ ]:
fit_line(group = a[2],test_size = 0.1)

In [ ]:
def anal(i,preds,y_oof,log = False):
    if log:
        test['pm2_5'] = np.exp(preds)
    else:
        test['pm2_5'] = preds

    all_data = pd.concat([train,test])
    a = all_data.groupby('site_latitude').get_group(all_data.site_latitude.unique()[i])[['date','pm2_5','Train']]
    y1 = a.loc[a.Train == 1].sort_values(by = 'date')['pm2_5'].values
    y2 = a.loc[a.Train == 0].sort_values(by = 'date')['pm2_5'].values
    x1 = range(len(y1))
    x2 = range(len(y1),len(y1)+len(y2))
    if log :
        y3 = np.exp(y_oof[a.loc[a.Train == 1].index])
    else:
        y3 = y_oof[a.loc[a.Train == 1].index]
    plt.plot(x1,y1,label = 'true')
    plt.plot(x1,y3)
    plt.plot(x2,y2)

In [ ]:
# anal(1,y_predicted_cat)
# anal(7,y_predicted_lgb,y_oof_lgb)
# test.groupby('site_code')

# sns.distplot(test['pm2_5'])

# sns.distplot(train['pm2_5'])
# train.groupby('site_latitude')['site_longitude'].unique()
# train.groupby('site_code').apply(lambda )
# train.index = train.date

In [ ]:
# df.select_dtypes(exclude ='object')
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(df.drop('date',axis = 1).values)

In [ ]:
test.index = test.date

In [ ]:
test.sort_index().groupby('site_code').resample('D').mean().shape

In [ ]:
test.shape

In [ ]:
sub

In [ ]:
sub.to_csv('lgb_5.csv',index = False)

In [ ]:
from catboost import Pool, CatBoostRegressor
params_cat = {'iterations':1500,
        'learning_rate':0.01,
        'random_strength':0.1,
        'depth':8,
        'loss_function':'MAE',
        'eval_metric':'MAE',
        'verbose' : 100}
y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train.drop(drop_cols,axis = 1),
                                                                y,test.drop(drop_cols,axis = 1),params_cat,42,
                                                               True)